In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

learning_rate = 0.0005  # Reduced learning rate
batch_size = 16         # Increased batch size

class EnhancedCIFAR10CNN(nn.Module):
    def __init__(self):
        super(EnhancedCIFAR10CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 10)


    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 256 * 2 * 2)  # Adjusting the view to match the new layer dimensions
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
print(device)

cifar10_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))
])

cifar10_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=cifar10_transform)

batch_size = 8
cifar10_trainloader = torch.utils.data.DataLoader(cifar10_trainset, batch_size=batch_size, shuffle=True, num_workers=2 if cuda_available else 0)

cifar10_net = EnhancedCIFAR10CNN()
cifar10_net.to(device)

criterion_cifar10 = nn.CrossEntropyLoss()
optimizer_cifar10 = optim.Adam(cifar10_net.parameters(), lr=learning_rate)

validation_split = 0.2
shuffle_dataset = True
random_seed = 42

dataset_size = len(cifar10_trainset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(val_indices)

cifar10_trainloader = torch.utils.data.DataLoader(cifar10_trainset, batch_size=batch_size, sampler=train_sampler, num_workers=2 if cuda_available else 0)
cifar10_valloader = torch.utils.data.DataLoader(cifar10_trainset, batch_size=batch_size, sampler=validation_sampler, num_workers=2 if cuda_available else 0)

start_time_cifar10 = time.time()
for epoch in range(10):
    running_loss_cifar10 = 0.0
    for i, data in enumerate(cifar10_trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_cifar10.zero_grad()

        outputs = cifar10_net(inputs)
        loss = criterion_cifar10(outputs, labels)
        loss.backward()
        optimizer_cifar10.step()

        running_loss_cifar10 += loss.item()
        if i % 2000 == 1999:
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss_cifar10 / 2000:.3f}')
            running_loss_cifar10 = 0.0

print("Finished Training CIFAR-10")
end_time_cifar10 = time.time()
training_time_cifar10 = end_time_cifar10 - start_time_cifar10
print(f"Training time for CIFAR-10: {training_time_cifar10} seconds")
model_save_path = 'CNN-91.74.h5'
torch.save(cifar10_net.state_dict(), model_save_path)

print(f"Model saved to {model_save_path}")

cifar10_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=cifar10_transform)
cifar10_testloader = torch.utils.data.DataLoader(cifar10_testset, batch_size=batch_size, shuffle=False, num_workers=2 if cuda_available else 0)

def calculate_metrics(correct, total):
    accuracy = correct / total
    return accuracy

correct_predictions = 0
total_samples = 0

cifar10_net.eval()
with torch.no_grad():
    for data in cifar10_testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = cifar10_net(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = calculate_metrics(correct_predictions, total_samples)

print(f'Test Accuracy: {accuracy*100:.2f}%')


cuda
Files already downloaded and verified
Epoch 1, Batch 2000, Loss: 1.722
Epoch 1, Batch 4000, Loss: 1.461
Epoch 2, Batch 2000, Loss: 1.282
Epoch 2, Batch 4000, Loss: 1.195
Epoch 3, Batch 2000, Loss: 1.097
Epoch 3, Batch 4000, Loss: 1.063
Epoch 4, Batch 2000, Loss: 0.994
Epoch 4, Batch 4000, Loss: 0.980
Epoch 5, Batch 2000, Loss: 0.920
Epoch 5, Batch 4000, Loss: 0.904
Epoch 6, Batch 2000, Loss: 0.861
Epoch 6, Batch 4000, Loss: 0.850
Epoch 7, Batch 2000, Loss: 0.815
Epoch 7, Batch 4000, Loss: 0.808
Epoch 8, Batch 2000, Loss: 0.763
Epoch 8, Batch 4000, Loss: 0.769
Epoch 9, Batch 2000, Loss: 0.730
Epoch 9, Batch 4000, Loss: 0.740
Epoch 10, Batch 2000, Loss: 0.715
Epoch 10, Batch 4000, Loss: 0.703
Finished Training CIFAR-10
Training time for CIFAR-10: 661.3623766899109 seconds
Model saved to model.pth
Files already downloaded and verified
Test Accuracy: 77.48%
